### 1. Import required packages

In [16]:
import MetaTrader5 as mt5
import time
import numpy as np
import pandas as pd

### 2. Initialise MT5 binding

In [17]:
mt5.initialize("C:\\Program Files\\Pepperstone MetaTrader 5\\terminal64.exe")

True

### 3. Basic money management

#### BUY order with take profit & stop loss

In [18]:
# Variables
volume = 0.1
symbol = "EURUSD_SB"
deviation = 10

# Extract symbol point
point = mt5.symbol_info(symbol).point

# Find the filling mode of symbol
#filling_type = mt5.symbol_info(symbol).filling_mode

# Create request
request = {
    "action": mt5.TRADE_ACTION_DEAL,
    "symbol": symbol,
    "volume": volume,
    "type": mt5.ORDER_TYPE_BUY,
    "deviation": deviation,
    "tp": mt5.symbol_info_tick(symbol).ask + 100*point,
    "sl": mt5.symbol_info_tick(symbol).ask - 100*point, 
    "type_filling": mt5.ORDER_FILLING_IOC, # OR filling_type
    "type_time": mt5.ORDER_TIME_GTC,
}

# Test order
mt5.order_check(request).comment

'Done'

#### SELL order with take profit & stop loss

In [19]:
# Variables
volume = 0.1
symbol = "EURUSD_SB"
deviation = 10

# Extract symbol point
point = mt5.symbol_info(symbol).point

# Find the filling mode of symbol
#filling_type = mt5.symbol_info(symbol).filling_mode

# Create request
request = {
    "action": mt5.TRADE_ACTION_DEAL,
    "symbol": symbol,
    "volume": volume,
    "type": mt5.ORDER_TYPE_SELL,
    "price": mt5.symbol_info_tick(symbol).bid,
    "deviation": deviation,
    "tp": mt5.symbol_info_tick(symbol).ask - 100 * point,
    "sl": mt5.symbol_info_tick(symbol).ask + 100 * point, 
    "type_filling": mt5.ORDER_FILLING_IOC, # OR filling_type
    "type_time": mt5.ORDER_TIME_GTC,
}

mt5.order_check(request).comment

'Done'

#### Find stop loss and take profit levels for a specific risk percentage

In [20]:
# Variables
symbol = "EURUSD_SB"
buy = True
risk  = 0.01
risk_reward_ratio = 2

# Extract account leverage
leverage = mt5.account_info().leverage

# Compute the price
price = mt5.symbol_info(symbol).ask

# Extract the number of decimals
nb_decimal = str(price)[::-1].find(".")


# Compute the variations in percentage
var_down = risk/leverage
var_up = (risk * risk_reward_ratio)/leverage


# Find the TP and SL threshold in absolute price
if buy:
    price = mt5.symbol_info(symbol).ask

    # Compute the variations in absolute price
    price_var_down = var_down*price
    price_var_up = var_up * price

    tp = np.round(price + price_var_up, nb_decimal)
    sl = np.round(price - price_var_down, nb_decimal)

else:

    price = mt5.symbol_info(symbol).bid

    # Compute the variations in absolute price
    price_var_down = var_down*price
    price_var_up = var_up * price

    tp = np.round(price - price_var_up, nb_decimal)
    sl = np.round(price + price_var_down, nb_decimal)

print(f"PRICE: {price} \t Take Profit: {tp} \t Stop Loss: {sl}")

PRICE: 1.10352 	 Take Profit: 1.10426 	 Stop Loss: 1.10315


#### Find the volume based on account capital

In [21]:
# Variables
symbol = "EURUSD_SB"

# Read account balance
capital = 6000 # mt5.account_info().balance
print(f"INVESTED CAPITAL: {capital}")

leverage = mt5.account_info().leverage
print(f"LEVERAGE: {leverage}")

invested_capital = capital  * leverage
print(f"INVESTED CAPITAL LEVERAGED: {invested_capital}")

trade_size = mt5.symbol_info(symbol).trade_contract_size
print(f"TRADE SIZE: {trade_size}")

price = (mt5.symbol_info(symbol).ask + mt5.symbol_info(symbol).bid)/2
print(f"PRICE: {price}")

lot_size = invested_capital / trade_size / price
print(f"LOT SIZE: {lot_size}")

min_lot = mt5.symbol_info(symbol).volume_min
print(f"MIN LOT: {min_lot}")

max_lot = mt5.symbol_info(symbol).volume_max
print(f"MAX LOT: {max_lot}")


if min_lot<lot_size:
    number_decimal = str(min_lot)[::-1].find(".")
    print(f"NUMBER DECIMAL: {number_decimal}")

    if number_decimal>0:
        lot_size_rounded = np.round(lot_size, number_decimal)
        print(f"LOT SIZE ROUNDED: {lot_size_rounded}")

        if lot_size < lot_size_rounded:
            lot_size_rounded = np.round(lot_size_rounded - min_lot, number_decimal)
            print(f"LOT DOWN ROUNDED: {lot_size_rounded}")

    else:
        number_size_lot =  len(str(min_lot))

        lot_size_rounded = int(np.round(lot_size, -number_size_lot))

        if lot_size < lot_size_rounded:
            lot_size_rounded = int(np.round(lot_size_rounded - number_size_lot, - number_size_lot))
            
    if lot_size_rounded>max_lot:
        lot_size_rounded = max_lot
        
    print(f"GOOD SIZE LOT: {lot_size_rounded}")
else: 
    print("Invested capital is too small to be able to place an order")

INVESTED CAPITAL: 6000
LEVERAGE: 30
INVESTED CAPITAL LEVERAGED: 180000
TRADE SIZE: 10000.0
PRICE: 1.103455
LOT SIZE: 16.312400596308866
MIN LOT: 0.1
MAX LOT: 1000.0
NUMBER DECIMAL: 1
LOT SIZE ROUNDED: 16.3
GOOD SIZE LOT: 16.3


### 3. Advanced money management

#### Retrieve current positions

In [25]:
mt5.positions_get()

(TradePosition(ticket=10964307, time=1703891675, time_msc=1703891675343, time_update=1703891675, time_update_msc=1703891675343, type=0, magic=0, identifier=10964307, reason=3, volume=0.1, price_open=1.10362, sl=0.0, tp=0.0, price_current=1.10351, swap=0.0, profit=-0.11, symbol='EURUSD_SB', comment='', external_id=''),
 TradePosition(ticket=10964308, time=1703891679, time_msc=1703891679251, time_update=1703891679, time_update_msc=1703891679251, type=1, magic=0, identifier=10964308, reason=3, volume=0.1, price_open=1.10351, sl=0.0, tp=0.0, price_current=1.10362, swap=0.0, profit=-0.11, symbol='EURUSD_SB', comment='', external_id=''))

In [26]:
def get_current_positions():
    """ Return the current positions. Position=0 --> Buy """    
    # Define the name of the columns that we will create
    columns = ["ticket", "position", "symbol", "volume", "magic", "profit", "price", "tp", "sl","trade_size"]

    # Go take the current open trades
    list_current = mt5.positions_get()

    # Create a empty dataframe
    summary = pd.DataFrame()

    # Loop to add each row in dataframe
    for element in list_current:
        element_pandas = pd.DataFrame([element.ticket, element.type, element.symbol, element.volume, element.magic,
                                       element.profit, element.price_open, element.tp,
                                       element.sl, mt5.symbol_info(element.symbol).trade_contract_size],
                                      index=columns).transpose()
        summary = pd.concat((summary, element_pandas), axis=0)
    
    try:
        summary["profit %"] = summary.profit / (summary.price * summary.trade_size * summary.volume)
        summary = summary.reset_index(drop=True)
    except:
        pass
    return summary

In [29]:
get_current_positions()

,ticket,position,symbol,volume,magic,profit,price,tp,sl,trade_size,profit %
0,10964330,0,EURUSD_SB,0.1,0,-0.14,1.10356,0.0,0.0,10000.0,-0.000127
1,10964331,1,EURUSD_SB,0.1,0,-0.12,1.10343,0.0,0.0,10000.0,-0.000109


#### Implement Trailing Stop Loss

In [32]:
def trailing_stop_loss():
    global max_price, min_price, summary
    
    # Extract the current open positions
    summary = get_current_positions()
    
    # Verification: Is there any open position?
    if summary.shape[0] > 0:
        for i in range(summary.shape[0]):
            
            # Extract information
            row = summary.iloc[i]
            symbol = row["symbol"]

            """ CASE 1: Change dynamicly the stop loss for a BUY ORDER """
            # Trailing stop loss for a buy order
            if row["position"] == 0:
               
                if symbol not in max_price.keys():
                    max_price[symbol]=row["price"]
                
                # Extract current price 
                current_price = (mt5.symbol_info(symbol).ask + mt5.symbol_info(symbol).bid ) / 2
                
                #Compute distance between current price an max price
                from_sl_to_curent_price = current_price - row["sl"]
                from_sl_to_max_price = max_price[symbol] - row["sl"]
                
                
                # If current price is greater than preivous max price --> new max price
                if current_price > max_price[symbol]:
                    max_price[symbol] = current_price
                    
                
                # Find the difference between the current minus max 
                if from_sl_to_curent_price > from_sl_to_max_price:
                    difference = from_sl_to_curent_price - from_sl_to_max_price

                    # Set filling mode
                    filling_type = mt5.symbol_info(symbol).filling_mode

                    # Set the point
                    point = mt5.symbol_info(symbol).point

                    # Change the sl
                    request = {
                    "action": mt5.TRADE_ACTION_SLTP,
                    "symbol": symbol,
                    "position": row["ticket"],
                    "volume": row["volume"],
                    "type": mt5.ORDER_TYPE_BUY,
                    "price": row["price"],
                    "sl": row["sl"] + difference,
                    "type_filling": mt5.ORDER_FILLING_IOC,
                    "type_time": mt5.ORDER_TIME_GTC,
                    }
                    
                    information = mt5.order_send(request)
                    print(information)
                    
            
            """ CASE 2: Change dynamicly the stop loss for a SELL ORDER """
            # Trailing stop loss for a sell order
            if row["position"] == 1:
                
                if symbol not in min_price.keys():
                    min_price[symbol]=row["price"]
                    
                # Extract current price 
                current_price = (mt5.symbol_info(symbol).ask + mt5.symbol_info(symbol).bid ) / 2
                
                
                
                #Compute distance between current price an max price
                from_sl_to_curent_price = row["sl"] - current_price
                from_sl_to_min_price = row["sl"] - min_price[symbol]
                
                 # If current price is greater than preivous max price --> new max price
                if current_price < min_price[symbol]:
                    min_price[symbol] = current_price
                                
                    
                # Find the difference between the current minus max 
                if from_sl_to_curent_price > from_sl_to_min_price:
                    difference = from_sl_to_curent_price - from_sl_to_min_price 

                    # Set filling mode
                    filling_type = mt5.symbol_info(symbol).filling_mode

                    # Set the point
                    point = mt5.symbol_info(symbol).point

                    # Change the sl
                    request = {
                    "action": mt5.TRADE_ACTION_SLTP,
                    "symbol": symbol,
                    "position": row["ticket"],
                    "volume": row["volume"],
                    "type": mt5.ORDER_TYPE_SELL,
                    "price": row["price"],
                    "sl": row["sl"] - difference,
                    "type_filling": mt5.ORDER_FILLING_IOC,
                    "type_time": mt5.ORDER_TIME_GTC,
                    }
                    
                
                    information = mt5.order_send(request)
                    print(information)

(1, 'Success')